In [ ]:
from google.colab import files
files.upload()

import os
os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json



Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-dataset

Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-dataset
License(s): MIT


In [ ]:
# Check if the file exists before attempting to open it
import zipfile
import os
if os.path.exists("fashion-product-images-dataset.zip"):
    with zipfile.ZipFile("fashion-product-images-dataset.zip", 'r') as zip_ref:
        zip_ref.extractall("fashion_data")
else:
    print("Error: 'fashion-product-images-dataset.zip' not found. Please ensure the download was successful.")



In [ ]:
import pandas as pd
df = pd.read_csv("/content/fashion_data/fashion-dataset/fashion-dataset/styles.csv", on_bad_lines='skip')

In [ ]:
# Get unique subCategory labels
unique_labels = df['subCategory'].dropna().unique()

# Count of unique labels
num_labels = len(unique_labels)

print(f"Number of unique masterCategory labels: {num_labels}")
print("Labels:", unique_labels)


Number of unique masterCategory labels: 45
Labels: ['Topwear' 'Bottomwear' 'Watches' 'Socks' 'Shoes' 'Belts' 'Flip Flops'
 'Bags' 'Innerwear' 'Sandal' 'Shoe Accessories' 'Fragrance' 'Jewellery'
 'Lips' 'Saree' 'Eyewear' 'Nails' 'Scarves' 'Dress'
 'Loungewear and Nightwear' 'Wallets' 'Apparel Set' 'Headwear' 'Mufflers'
 'Skin Care' 'Makeup' 'Free Gifts' 'Ties' 'Accessories' 'Skin'
 'Beauty Accessories' 'Water Bottle' 'Eyes' 'Bath and Body' 'Gloves'
 'Sports Accessories' 'Cufflinks' 'Sports Equipment' 'Stoles' 'Hair'
 'Perfumes' 'Home Furnishing' 'Umbrellas' 'Wristbands' 'Vouchers']


In [ ]:
import os
import shutil
from tqdm import tqdm

# Create a root dataset folder
os.makedirs("fashion_data_sub", exist_ok=True)

# Create subfolders for each subCategory label
for label in df['subCategory'].unique():
    os.makedirs(os.path.join("fashion_data_sub", label), exist_ok=True)

# Move images into label folders
source_dir = "/content/fashion_data/fashion-dataset/images"  # Folder from the dataset where original images are stored
for idx, row in tqdm(df.iterrows(), total=len(df)):
    img_name = '{}.jpg'.format(row['id'])
    label = row['subCategory']
    src = os.path.join(source_dir, img_name)
    dst = os.path.join("fashion_data_sub", label, img_name)
    if os.path.exists(src):
        shutil.copy(src, dst)

100%|██████████| 44424/44424 [01:22<00:00, 540.64it/s]


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


full_dataset = datasets.ImageFolder("fashion_data_sub", transform=transform)


total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size


train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)


In [ ]:
import torch
import torch.nn as nn
import timm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load EfficientNet with 45 output classes
model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=45)
model = model.to(device)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
import torch
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=10, class_names=None):
    best_f1 = 0.0

    for epoch in range(epochs):
        # ---- Training ----
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        f1 = f1_score(all_labels, all_preds, average='weighted')

        # ---- Save Best Model ----
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"✅ Best model updated at Epoch {epoch+1} | F1: {f1:.4f}")

        # ---- Confusion Matrix ----
        cm = confusion_matrix(all_labels, all_preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names if class_names else np.unique(all_labels),
                    yticklabels=class_names if class_names else np.unique(all_labels))
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.title(f'Confusion Matrix - Epoch {epoch+1}')
        plt.tight_layout()
        plt.savefig(f'confusion_matrix_epoch_{epoch+1}.png')
        plt.close()

        # ---- Logging ----
        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | F1 Score: {f1:.4f}")


Epoch 1 | Loss: 0.3258
Epoch 2 | Loss: 0.1093
Epoch 3 | Loss: 0.0663
Epoch 4 | Loss: 0.0466
Epoch 5 | Loss: 0.0331
Epoch 6 | Loss: 0.0239
Epoch 7 | Loss: 0.0238
Epoch 8 | Loss: 0.0186
Epoch 9 | Loss: 0.0155
Epoch 10 | Loss: 0.0164


In [ ]:
train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    epochs=10
)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F
import copy

In [ ]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='macro')
print(f"\n📊 Test Accuracy: {acc:.4f}, F1 (macro): {f1:.4f}")


📊 Test Accuracy: 0.9682, F1 (macro): 0.8164


In [ ]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f"\n📊 Test Accuracy: {acc:.4f}, F1 (weighted): {f1:.4f}")